# Test of ground station acceleration contribution in NEAR's ΔV

Antreasian & Guinn 1998 gave the loss of signal (LOS) at Goldstone and acquisition of signal (AOS) by Canberra were
given relative to the computed perigee time, and only to minute precision. The velocity lags Δv ≡ -ar/c are
sensitive to these times, however.

Automated fits are explored below for the precise LOS and AOS times that would exactly account for NEAR's velocity
anomaly, as a further test against the now recovered original tracking data. 


In [1]:
from astropy import units as u
from astropy import constants as const
from astropy.coordinates import solar_system_ephemeris

from poliastro.ephem import Ephem
from poliastro.bodies import Earth
from poliastro.frames import Planes

import datetime
from lmfit import Parameters, minimize, fit_report

import sys
sys.path.append('../')

from sim.stations import dss25, dss34
from sim.tracking import Tracking
from sim.util import make_epochs, horizons_range_rate_accel

In [2]:
solar_system_ephemeris.set("de440")
near_perigee = Tracking.NEAR_PERIGEE.value
goldstone_end = Tracking.NEAR_GOLDSTONE_END.value
canberra_start = Tracking.NEAR_CANBERRA_START.value
ref_dv = {
    'AIAA': 13.0*u.mm/u.s, # From Antreasian & Guinn, AIAA 1998
    'PRL': 13.46*u.mm/u.s, # From Anderson et al, PRL 2008
}

initial_offsets = Parameters()
initial_offsets.add('los', min=-300, max=+300, value=0)
initial_offsets.add('aos', min=-300, max=+300, value=0)

los_epochs = make_epochs(goldstone_end - 10*u.min, goldstone_end + 10*u.min, 1*u.min)
aos_epochs = make_epochs(canberra_start - 10*u.min, canberra_start + 10*u.min, 1*u.min)

los_ephem = Ephem.from_horizons("NEAR", los_epochs, attractor=Earth, plane=Planes.EARTH_EQUATOR)
aos_ephem = Ephem.from_horizons("NEAR", aos_epochs, attractor=Earth, plane=Planes.EARTH_EQUATOR)


def hms(tdelta):
    return str(tdelta.to_datetime())


def report(los_epoch, los_dv, aos_epoch, aos_dv, ref_name, ref_dv, e_dv):

    los_gap = near_perigee - los_epoch
    aos_gap = aos_epoch - near_perigee
    total_gap = aos_epoch - los_epoch
    e_pc = (100*e_dv/ref_dv)

    print(f'{ref_name} ({ref_dv}): error {e_dv << u.mm/u.s:.3e} ={e_pc:.4f}%')
    print('LOS', los_epoch, f'(-{hms(los_gap)}) {los_dv << (u.mm/u.s):.3f}')
    print('AOS', aos_epoch, f'(+{hms(aos_gap)}) {aos_dv << (u.mm/u.s):.3f}')
    print(f'Total gap: {hms(total_gap)}\n')


def geocentric_full(offsets, ref_dv, ref_name, verbose=False):

    los_epoch = goldstone_end + offsets['los']*u.s
    aos_epoch = canberra_start + offsets['aos']*u.s

    los_r, los_rr, los_ra, los_rs = dss25.range_rate_accel(los_ephem, los_epoch)
    aos_r, aos_rr, aos_ra, aos_rs = dss34.range_rate_accel(aos_ephem, aos_epoch)

    los_dv = -los_ra*los_r/const.c << u.mm/u.s
    aos_dv = -aos_ra*aos_r/const.c << u.mm/u.s
    dv = los_dv + aos_dv
    e_dv = dv - ref_dv

    if verbose:
        report(los_epoch, los_dv, aos_epoch, aos_dv, ref_name, ref_dv, e_dv)

    return abs(e_dv)


def geocentric_nostation(offsets, ref_dv, ref_name, verbose=False):

    los_epoch = goldstone_end + offsets['los']*u.s
    aos_epoch = canberra_start + offsets['aos']*u.s

    los_r, los_rr, los_ra, los_rs = dss25.range_rate_accel(los_ephem, los_epoch)
    aos_r, aos_rr, aos_ra, aos_rs = dss34.range_rate_accel(aos_ephem, aos_epoch)

    los_dv = -(los_ra+los_rs)*los_r/const.c << u.mm/u.s
    aos_dv = -(aos_ra+aos_rs)*aos_r/const.c << u.mm/u.s
    dv = los_dv + aos_dv
    e_dv = dv - ref_dv

    if verbose:
        report(los_epoch, los_dv, aos_epoch, aos_dv, ref_name, ref_dv, e_dv)

    return abs(e_dv)


def find(func):
    for src in ref_dv:
        result = minimize(func, initial_offsets, args=(ref_dv[src], ), kws={'ref_name': src}, method='nelder')
        #print(fit_report(result))
        func(result.params, ref_dv[src], src, verbose=True)

In [3]:
print(f'Perigee: {near_perigee}  # Horizons (AIAA: 55.6)')
report(goldstone_end, 2.51*u.mm/u.s, canberra_start, 11.13*u.mm/u.s,
       'NAECON', ref_dv['PRL'], 13.64*u.mm/u.s - ref_dv['PRL'])

Perigee: 1998-01-23 07:22:57.000  # Horizons (AIAA: 55.6)
NAECON (13.46 mm / s): error 1.800e-01 mm / s =1.3373%
LOS 1998-01-23 06:14:55.600 (-1:09:04.584535) 2.510 mm / s
AOS 1998-01-23 09:53:55.600 (+2:29:55.415463) 11.130 mm / s
Total gap: 3:39:00



In [4]:
find(geocentric_full)

AIAA (13.0 mm / s): error 3.037e-05 mm / s =0.0002%
LOS 1998-01-23 06:17:31.653 (-1:06:28.531093) 1.891 mm / s
AOS 1998-01-23 09:53:29.680 (+2:29:29.495492) 11.109 mm / s
Total gap: 3:35:58.026586

PRL (13.46 mm / s): error 1.363e-05 mm / s =0.0001%
LOS 1998-01-23 06:16:24.635 (-1:07:35.549923) 2.353 mm / s
AOS 1998-01-23 09:53:41.644 (+2:29:41.459305) 11.107 mm / s
Total gap: 3:37:17.009230



In [5]:
find(geocentric_nostation)

AIAA (13.0 mm / s): error -1.507e-04 mm / s =-0.0012%
LOS 1998-01-23 06:18:09.507 (-1:05:50.677813) 4.813 mm / s
AOS 1998-01-23 09:50:13.556 (+2:26:13.371239) 8.187 mm / s
Total gap: 3:32:04.049053

PRL (13.46 mm / s): error -2.258e-06 mm / s =-0.0000%
LOS 1998-01-23 06:17:12.113 (-1:06:48.071117) 5.271 mm / s
AOS 1998-01-23 09:51:16.391 (+2:27:16.206543) 8.189 mm / s
Total gap: 3:34:04.277662

